In [1]:
import math
import datetime
import numpy as np
import pickle

import methodsMLinterns
import customAutoencoder

Using TensorFlow backend.


In [2]:
stocks = ['DNB', 'NRG', 'CL', 'ANTM', 'NEE', 'PAYX', 'VAR', 'NI', 'MNST', 'JNJ', 'TGNA', 'NOV', 'FIS', 'BLK', 'HBI', 'NVDA', 'DLTR', 'MRO', 'EMN', 'AMT', 'FLR', 'IBM', 'BK', 'NFX', 'AGN', 'LRCX', 'DIS', 'LH', 'C', 'MNK']

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [4]:
features_1p4_extra =['aab', 'aac', 'aad', 'aae', 'aaf', 'aag', 'aah', 'abj', 'abm', 'abn', 'abo', 'abp', 'abq', 'abr', 'abs', 'abt', 'abu', 'abv', 'abw', 'abx', 'aby', 'abz', 'aca', 'acb', 'acc', 'acd', 'ace', 'acf', 'acr', 'acw', 'acx', 'acy', 'adi', 'adj', 'adl', 'ado', 'adp', 'adq', 'adr', 'ads', 'adt', 'adu', 'adv', 'adw', 'adx', 'ady', 'adz', 'aea', 'aeb', 'aec', 'aed', 'aee', 'aef', 'aeg', 'aeh', 'aei', 'aej', 'aek', 'ael', 'aem', 'aen', 'aeo', 'aep', 'aeq', 'aer', 'aes', 'aex', 'aey', 'aez', 'afa', 'afj', 'afl', 'afo', 'afp', 'afq', 'afr', 'afs', 'aft', 'afu', 'afv', 'afw', 'afx', 'afy', 'afz', 'aga', 'agb', 'agc', 'agd', 'age', 'agf', 'agg', 'agh', 'agi', 'agj', 'agk', 'agl', 'agm', 'agn', 'ago', 'agp', 'agq', 'agr', 'ags', 'agt', 'agu', 'agv', 'agw', 'agx', 'agy', 'ahf', 'ahg', 'ahh', 'ahi', 'ahj', 'ahk', 'ahl', 'ahm', 'ahn', 'aho', 'zhq', 'zhr', 'zhs', 'zht', 'zhu', 'zhv', 'zhw', 'ziy', 'zjb', 'zjc', 'zjd', 'zje', 'zjf', 'zjg', 'zjh', 'zji', 'zjj', 'zjk', 'zjl', 'zjm', 'zjn', 'zjo', 'zjp', 'zjq', 'zjr', 'zjs', 'zjt', 'zju', 'zkg', 'zkl', 'zkm', 'zkn', 'zkx', 'zky', 'zla', 'zld', 'zle', 'zlf', 'zlg', 'zlh', 'zli', 'zlj', 'zlk', 'zll', 'zlm', 'zln', 'zlo', 'zlp', 'zlq', 'zlr', 'zls', 'zlt', 'zlu', 'zlv', 'zlw', 'zlx', 'zly', 'zlz', 'zma', 'zmb', 'zmc', 'zmd', 'zme', 'zmf', 'zmg', 'zmh', 'zmm', 'zmn', 'zmo', 'zmp', 'zmy', 'zna', 'znd', 'zne', 'znf', 'zng', 'znh', 'zni', 'znj', 'znk', 'znl', 'znm', 'znn', 'zno', 'znp', 'znq', 'znr', 'zns', 'znt', 'znu', 'znv', 'znw', 'znx', 'zny', 'znz', 'zoa', 'zob', 'zoc', 'zod', 'zoe', 'zof', 'zog', 'zoh', 'zoi', 'zoj', 'zok', 'zol', 'zom', 'zon', 'zou', 'zov', 'zow', 'zox', 'zoy', 'zoz', 'zpa', 'zpb', 'zpc', 'zpd', 'zpe']
features_1p4_extra_a=[f for f in features_1p4_extra if f[0]=='a']
features_1p4_extra_z=[f for f in features_1p4_extra if f[0]=='z']
assert(len(features_1p4_extra_a)+len(features_1p4_extra_z)==len(features_1p4_extra))

In [ ]:
date_test_set = datetime.date(2016, 1, 1)

clf_portfolio_dic = methodsMLinterns.ClassificationPortfolio(stocks=stocks, minutes_forward=30)
clf_portfolio_dic.loadData()
clf_portfolio_dic.cleanUpData(features_1p4_extra)
clf_portfolio_dic.getTrainTestSetDate(date_test_set)

### Group together all the stocks

In [7]:
def prepareData(features):
    X_train = np.array([], dtype=np.float64).reshape(0,len(features))
    y_train = np.array([], dtype=np.float64).reshape(0,1)
    X_test = np.array([], dtype=np.float64).reshape(0,len(features))
    y_test = np.array([], dtype=np.float64).reshape(0,1)


    for k, stock in enumerate(clf_portfolio_dic.stocks):
        name = stock + str(clf_portfolio_dic.minutes_forward)
        if k==0:
            X_train, y_train = clf_portfolio_dic.X_train_dic[name][features].as_matrix(),(clf_portfolio_dic.y_train_dic[name]+1)/2
            X_test, y_test = clf_portfolio_dic.X_test_dic[name][features].as_matrix(), (clf_portfolio_dic.y_test_dic[name]+1)/2
        else:
            X_train = np.concatenate((X_train,clf_portfolio_dic.X_train_dic[name][features].as_matrix()),axis=0)
            y_train = np.concatenate((y_train,(clf_portfolio_dic.y_train_dic[name]+1)/2),axis=0)
            X_test = np.concatenate((X_test,clf_portfolio_dic.X_test_dic[name][features].as_matrix()),axis=0)
            y_test = np.concatenate((y_test,(clf_portfolio_dic.y_test_dic[name]+1)/2),axis=0)

    print(X_train.shape)
    # Transform to one hot vectors
    y_t = np.zeros((y_train.shape[0], 2))
    y_t[np.arange(y_train.shape[0]), y_train.astype('int32')] = 1
    y_train = y_t

    y_t = np.zeros((y_test.shape[0], 2))
    y_t[np.arange(y_test.shape[0]), y_test.astype('int32')] = 1
    y_test = y_t
    
    return X_train,y_train,X_test,y_test

### Load saved models

In [83]:
best_30_a = "A30S_(119,100,30)_d0_in0.1_r(0.1,0.1)_f0"
best_30_z = "A30S_(120,100,30)_d0.5_in0.1_r(0.01,0.01)_f0"
best_30_a_z = "A30S_(239,100,30)_d0.5_in0.1_r(0.01,0.01)_f0"
best_50_a = "A30S_(119,100,50)_d0.8_in0_r(0.01,0.01)_f0"
best_50_z = "A30S_(120,100,50)_d0.2_in0_r(0.01,0.01)_f1"
best_50_a_z = "A30S_(239,100,50)_d0.5_in0.1_r(0.01,0.01)_f0"
best_70_a = "A30S_(119,100,70)_d0.8_in0.1_r(0.01,0.01)_f0"
best_70_z = "A30S_(120,100,70)_d0.5_in0_r(0.01,0.01)_f1"
best_70_a_z = "A30S_(239,100,70)_d0.5_in0_r(0,0)_f1"
best_10_a_z = "A30S_(239,100,10)_d0.5_in0_r(0.01,0.01)_f0"
nameList = [best_30_a, best_30_z, best_30_a_z, best_50_a, best_50_z, best_50_a_z, best_70_a, best_70_z, best_70_a_z]
#7 3 6 0 1 4 5 8 2
nameList = [best_30_a_z,best_70_a_z,best_50_a_z,best_50_z,best_30_z,best_10_a_z]# , , , , , , , , ]

In [84]:
ensemble = customAutoencoder.AutoencoderEnsemble(nameList)
ensemble.loadModels()

In [35]:
features = features_1p4_extra
X_train,y_train,X_test,y_test = prepareData(features)

(122607, 239)


In [36]:
hardVoting_train,softVoting_train,productVoting_train,probas_train = ensemble.predict(X_train)
hardVoting_test,softVoting_test,productVoting_test,probas_test = ensemble.predict(X_test)

In [37]:
probas_test = np.argmax(probas_test,axis=2)
print(probas_test)

[[0 0 0 ..., 1 1 0]
 [1 1 1 ..., 1 1 0]
 [0 1 1 ..., 1 1 0]]


In [29]:
truthTable = (probas_test == np.repeat(np.argmax(y_test,axis=1)[None,:],9,axis=0)).astype('int32')
print(truthTable)

[[1 0 0 ..., 1 1 0]
 [0 1 0 ..., 1 1 0]
 [1 1 1 ..., 1 1 0]
 ..., 
 [1 0 0 ..., 1 1 0]
 [0 1 0 ..., 1 1 0]
 [0 0 0 ..., 1 1 0]]


In [32]:
print(np.sum(truthTable,axis=1))
print(np.argsort(np.sum(truthTable,axis=1)))

[19709 19753 19909 19660 19779 19863 19704 19648 19881]
[7 3 6 0 1 4 5 8 2]


In [13]:
def correlation(vote1,vote2):
    return np.corrcoef(np.argmax(vote1,axis=1),np.argmax(vote2,axis=1))[0,1]
print(correlation(softVoting_train,hardVoting_train))
print(correlation(softVoting_train,productVoting_train))
print(correlation(productVoting_train,hardVoting_train))
print("==========")
print(correlation(softVoting_test,hardVoting_test))
print(correlation(softVoting_test,productVoting_test))
print(correlation(productVoting_test,hardVoting_test))

0.876227527437
0.999345759882
0.875608855414
0.865084224911
0.999282151154
0.864480244678


### Test on stocks separately

In [38]:
def testOnSeparateStocks(auto,feat,votingType='soft'):
    acc_train_autoencoder = np.zeros(clf_portfolio_dic.N_stocks)
    acc_test_autoencoder = np.zeros(clf_portfolio_dic.N_stocks)
    for k, stock in enumerate(clf_portfolio_dic.stocks):
        name = "%s30"%stock
        print(k,name)
    
        x_train = np.array(clf_portfolio_dic.X_train_dic[name][feat])
        x_test = np.array(clf_portfolio_dic.X_test_dic[name][feat])
        y_train = (clf_portfolio_dic.y_train_dic[name]+1)/2
        y_test = (clf_portfolio_dic.y_test_dic[name]+1)/2

        y_t = np.zeros((y_train.shape[0], 2))
        y_t[np.arange(y_train.shape[0]), y_train.astype('int32')] = 1
        y_train = y_t

        y_t = np.zeros((y_test.shape[0], 2))
        y_t[np.arange(y_test.shape[0]), y_test.astype('int32')] = 1
        y_test = y_t
    
        
        hardVoting_train, softVoting_train, productVoting_train, _ = ensemble.predict(x_train)
        hardVoting_test, softVoting_test, productVoting_test, _ = ensemble.predict(x_test)
        if votingType == 'soft':
            y_pred_train = softVoting_train
            y_pred_test = softVoting_test
        elif votingType == 'hard':
            y_pred_train = hardVoting_train
            y_pred_test = hardVoting_test
        elif votingType == 'product':
            y_pred_train = productVoting_train
            y_pred_test = productVoting_test
        
        acc_train = (100 * (np.argmax(y_pred_train,1) == np.argmax(y_train,1))).mean()
        acc_test = (100 * (np.argmax(y_pred_test,1) == np.argmax(y_test,1))).mean()
        acc_train_autoencoder[k] = acc_train
        acc_test_autoencoder[k] = acc_test
    return(acc_train_autoencoder,acc_test_autoencoder)

In [85]:
expList = []

for votingType in ['soft','hard']:#,'product']:
    name = "ensemble best %s voting"%votingType
    print(name)
    e = methodsMLinterns.ExperimentPerformance(
        methodName=name, 
        stocks=stocks, originalFeatures=features)
    
    acc_train_vec, acc_test_vec = testOnSeparateStocks(ensemble,features,votingType)
                    
    e.setTrainResults(acc_train_vec)
    e.setTestResults(acc_test_vec)
    expList.append(e)

ensemble best soft voting
0 DNB30
1 NRG30
2 CL30
3 ANTM30
4 NEE30
5 PAYX30
6 VAR30
7 NI30
8 MNST30
9 JNJ30
10 TGNA30
11 NOV30
12 FIS30
13 BLK30
14 HBI30
15 NVDA30
16 DLTR30
17 MRO30
18 EMN30
19 AMT30
20 FLR30
21 IBM30
22 BK30
23 NFX30
24 AGN30
25 LRCX30
26 DIS30
27 LH30
28 C30
29 MNK30
ensemble best hard voting
0 DNB30
1 NRG30
2 CL30
3 ANTM30
4 NEE30
5 PAYX30
6 VAR30
7 NI30
8 MNST30
9 JNJ30
10 TGNA30
11 NOV30
12 FIS30
13 BLK30
14 HBI30
15 NVDA30
16 DLTR30
17 MRO30
18 EMN30
19 AMT30
20 FLR30
21 IBM30
22 BK30
23 NFX30
24 AGN30
25 LRCX30
26 DIS30
27 LH30
28 C30
29 MNK30


In [86]:
print(len(expList))

2


In [87]:
with open("pickles/acc_autoencoder_logit_ensemble.p",'wb') as f:
    pickle.dump( expList, f, protocol=pickle.HIGHEST_PROTOCOL)